In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

for testing, this just initializes the dataloader, classifier, and patch model

In [ ]:
import torch
from train import Patch, CLIPClassifier, patch_loader

loader = patch_loader(split='validation', batch_size=4, num_samples=100, streaming=False)
batch = next(iter(loader))

In [ ]:
from train import CLIPClassifier
model = CLIPClassifier(deep=1024)

In [ ]:
tot = 0
for param in model.clip.parameters():
    tot += sum(param.shape)
print(tot)

In [ ]:
model = CLIPClassifier(deep=1024)
checkpoint = torch.load('checkpoints/imnet_1k.pt', map_location='cpu')
model.load_state_dict(checkpoint['model'])

In [ ]:
patch = Patch(model=model, target_label=965, patch_r=0.2, init_size=1024)

this gets the model predictions. it gets 3/4 correct here.

In [ ]:
logits = patch.forward(batch)
print(torch.argmax(logits, dim=-1))

you can see the images we're attacking here

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes[0][0].imshow(batch['pixel_values'][0])
axes[0][1].imshow(batch['pixel_values'][1])
axes[1][0].imshow(batch['pixel_values'][2])
axes[1][1].imshow(batch['pixel_values'][3])

axes[0][0].axis('off')
axes[0][1].axis('off')
axes[1][0].axis('off')
axes[1][1].axis('off')

plt.tight_layout()
plt.show()

here you can see what the input images look like with the patch applied

In [ ]:
import matplotlib.pyplot as plt

with torch.no_grad():
    patched = patch._apply_patch(batch['pixel_values'])

fig, axes = plt.subplots(2, 2, figsize=(8, 8))
axes[0][0].imshow(patched[0])
axes[0][1].imshow(patched[1])
axes[1][0].imshow(patched[2])
axes[1][1].imshow(patched[3])

axes[0][0].axis('off')
axes[0][1].axis('off')
axes[1][0].axis('off')
axes[1][1].axis('off')

plt.tight_layout()
plt.show()

## regressions

In [ ]:
!python train.py --config=configs/patch/toy.yml --device=cpu